### Librerías necesarias para el notebook

In [1]:
import pandas as pd
import ast

### Observaciones iniciales entre los tres datasets

In [2]:
df_applist = pd.read_csv("../data/app_list.csv")
print(f"app_list.csv tiene {df_applist.shape[0]} filas y {df_applist.shape[1]} columnas.")

app_list.csv tiene 29235 filas y 2 columnas.


In [3]:
df_steam = pd.read_csv("../data/steam_app_data.csv")
print(f"steam_app_data.csv tiene {df_steam.shape[0]} filas y {df_steam.shape[1]} columnas.")

steam_app_data.csv tiene 29235 filas y 39 columnas.


Podemos observar que los tres datasets tienen el mismo número de columnas. No obstante, antes de realizar unirlos, analizaremos los tres datasets por separado.

### Dataset *app_list.csv*

#### Exploración

Columnas:
- appid: código del juego. (int)
- name: nombre del juego. (str)

In [4]:
df_applist.head()

,appid,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force


Este dataframe es sencillo, puesto que solo existen dos columnas. A continuación miraremos si hay duplicados y/o valores nulos.

In [5]:
print(f"El dataframe tiene {df_applist['appid'].duplicated().sum()} videojuegos repetidos.")

El dataframe tiene 0 videojuegos repetidos.


In [6]:
print(df_applist.isnull().sum())

appid    0
name     5
dtype: int64


#### Conclusiones

- Este es el dataset que une la información de los otros dos (aunque solo trabajaremos con uno de estos dos). No es aconsejable cambiarlo, pero sí tenerlo en cuenta para las exploraciones de los otros ficheros.
- Puesto que tiene solo dos columnas, el fichero podría ser un diccionario donde la clave es *appid* y el valor *name*. Este nuevo fichero tendría la extension *.json*.

### Dataset *steam_app_data.csv*

#### Exploración

Columnas:
- **type**: tipo de producto. (str)

- **name**: nombre del videojuego. (str)
- **steam_appid**: código del videojuego. (int)
- **required_age**: edad mínima para jugar (int)
- **is_free**: ¿el videojuego es gratuito? (bool)
- **controller_support**: ¿se puede jugar al videojuego con mando? (bool) 
- **dlc**: contenido de videojuego descargable, si es que tiene. (list[int])
- **detailed_description**: descripción detallada del videojuego. (str)
- **about_the_game**: sinopsis del videojuego. (str)
- **short_description**: descripción detallada del videojuego. (str)
- **fullgame**: ¿el videojuego está completo? (bool)
- **supported_languages**: idiomas del videojuego. (str) *
- **header_image**: URL de la imagen de encabezado en Steam. (str)
- **website**: página web oficial del videojuego. (str)
- **pc_requirements**: hardware y software necesario para jugar al videojuego en Windows. (dict) *
- **mac_requirements**: hardware y software necesario para jugar al videojuego en Mac. (dict) *
- **linux_requirements**: hardware y software necesario para jugar al videojuego en Linux. (dict) *
- **legal_notice**: copyright/derechos de autor. (str) *
- **drm_notice**: Digital Rights Management. (str) *
- **ext_user_account_notice**: otros sistemas de cuentas ajenos a Steam para iniciar sesion en el videojuego. (str)
- **developers**: desarrolladores del videojuego. (list[str])
- **publishers**: publicadores del videojuego. (list[str])
- **demos**: demos, si es que tiene. (dict)
- **price_overview**: precio del videojuego en GBP, teniendo en cuenta descuentos. (dict)
- **packages**: código de los paquetes de ofertas en los que esté el videojuego. (list)
- **package_groups**: paquetes de ofertas en los que esté el videojuego. (list[dict])
- **platforms**: sistemas operativos en los que el videojuego está disponible. (dict)
- **metacritic**: puntuación de la página web *metacritic* junto con el enlace a la página correspondiente. (dict)
- **reviews**: comentarios de páginas web. (str) * 
- **categories**: categorías a las que pertenece el videojuego. (dict)
- **genres**: géneros a los que pertenece el videojuego. (dict)
- **screenshots**: capturas de pantalla del videojuego. (dict)
- **movies**: tráiler del juego. (dict)
- **recommendations**: total de recomendaciones por parte de los usuarios (dict)
- **achievements**: logros de Steam del videojuego. (dict)
- **release_date**: fecha de publicación en mes y año, también se distingue si se publicará en el futuro. (dict)
- **support_info**: página web y correo de soporte. (dict)
- **background**: imagen de fondo del videojuego en la página de Steam. (str)
- **content_descriptors**: información sobre contenido no apto para todos los públicos y notas al respecto. (dict)

*: El contenido tiene a veces código HTML que deberían ser caracteres vacíos ('').


Vemos que no todas las columnas nos podrían servir para un análisis posterior. Decidimos eliminar las siguientes columnas por los siguientes motivos:
- **type**: serviría para definir si se trata de un videojuego, DLC o demo, pero en los tres datasets solo hay información de los videojuegos. Existen códigos que hacen referencia a las demos o DLCs, pero no hay información al respecto en ninguno de los tres datasets.

- **fullgame**: solo hay valores nulos. Esto se debe a que un videojuego para ordenador no acostumbra a considerarse terminado, puesto que los sistemas operativos siguen actualizándose y los desarrolladores tienen que estar atentos a qualquier *bug* a medida que tanto el hardware como el software cambia.
- **header_image, screenshots, movies y background**: son columnas que hacen referencia a vídeos y fotos. Para este proyecto esta información no nos será útil y la descartaremos para así simplificar un poco el contenido final.
- Los diccionarios de la columna **platforms** tienen la estructura {'windows':bool,'mac':bool,'linux':bool}. Esta columna resulta redundante, puesto que **pc_requirements**, **mac_requirements**, **linux_requirements** ofrecen información más detallada sobre si se puede jugar al videojuego en un sistema operativo u otro. Que el valor sea nulo en qualquiera de las tres columnas indicará que el videojuego no está disponible para ese sistema operativo.

In [7]:
pd.set_option('display.max_columns', None) # Puesto que el dataframe tiene muchas columnas, quitamos el límite de visualización de columnas 
df_steam = df_steam.drop(['type','fullgame','header_image','screenshots','movies','background', 'platforms'], axis=1)
print(f"El dataframe tiene ahora {df_steam.shape[1]} columnas")
df_steam.head(3)

El dataframe tiene ahora 32 columnas


,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,supported_languages,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,metacritic,reviews,categories,genres,recommendations,achievements,release_date,support_info,content_descriptors
0,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 719, 'final': 7...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 65735},{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...","{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 2802},{'total': 0},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}","{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",{'total': 1992},{'total': 0},"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}","{'ids': [], 'notes': None}"


Miramos de qué tipo se consideran las columnas del dataframe.

In [8]:
df_steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29235 entries, 0 to 29234
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     29234 non-null  object 
 1   steam_appid              29235 non-null  int64  
 2   required_age             29086 non-null  float64
 3   is_free                  29086 non-null  object 
 4   controller_support       5998 non-null   object 
 5   dlc                      4975 non-null   object 
 6   detailed_description     29060 non-null  object 
 7   about_the_game           29060 non-null  object 
 8   short_description        29060 non-null  object 
 9   supported_languages      29072 non-null  object 
 10  website                  19252 non-null  object 
 11  pc_requirements          29086 non-null  object 
 12  mac_requirements         29086 non-null  object 
 13  linux_requirements       29086 non-null  object 
 14  legal_notice          

30 de 32 columnas se consideran objetos.

***Observaciones para la limpieza***:
- **name**, **detailed_description**, **about_the_game**, **short_description**, **website**, **legal_notice**, **drm_notice** y **ext_user_account_notice** deberían ser cadenas de texto (*str*) en el dataframe.
- **price_overview**, **metacritic**, **recommendations**, **achievements** deberían ser ser valores numéricos en el dataframe. En el caso de **price_overview** tendría que ser *float* mientras que los otros tres deberían ser *int*.
- El diccionario **release_date** tiene la estructura {'coming_soon':bool,'date':'day month, year'}. La columna podría ser *datetime*.

Antes de observar valores nulos, miramos si hay algún videojuego repetido. Lo comprobamos porque con el *steam_appid* porque se supone que es único.

In [9]:
print(f"El dataframe tiene {df_steam['steam_appid'].duplicated().sum()} videojuegos repetidos.")

El dataframe tiene 7 videojuegos repetidos.


Entonces, eliminamos duplicados antes de nada.

In [10]:
df_steam_unique = df_steam.drop_duplicates()
print(f"El dataframe tiene {df_steam_unique['steam_appid'].duplicated().sum()} videojuegos repetidos.")

El dataframe tiene 0 videojuegos repetidos.


Efectivamente, ahora no hay videojuegos duplicados. Ahora miramos los valores nulos del los datos.

In [11]:
print(df_steam_unique.isnull().sum())

name                           1
steam_appid                    0
required_age                 149
is_free                      149
controller_support         23230
dlc                        24253
detailed_description         175
about_the_game               175
short_description            175
supported_languages          163
website                     9979
pc_requirements              149
mac_requirements             149
linux_requirements           149
legal_notice               19164
drm_notice                 29070
ext_user_account_notice    28716
developers                   264
publishers                   149
demos                      27089
price_overview              3712
packages                    3370
package_groups               149
metacritic                 26253
reviews                    23323
categories                   714
genres                       196
recommendations            22504
achievements                2381
release_date                 149
support_in

Vemos que en la columna *name* hay un solo valor nulo. Con esto, 4 de 5 (o todos) casos de valores nulos del fichero *app_list.csv* se resuelven.
Observamos que en distintas columnas hay el mísmo número de nulos (149).
Sospechamos que hay 149 filas con los valores nulos en todas las columnas menos en *name* y *steam_appid*. Para comprobarlo, filtraremos de este modo:

In [12]:
no_nan = ["name", "steam_appid"] # columnas que sí que tendrán valores
cols = df_steam_unique.columns.difference(no_nan) # selección de todas las columnas menos las que tendrán valores
mask = df_steam_unique[cols].isna().all(axis=1) # de todo el dataframe, se filtraran las líneas con las condiciones estipuladas en el markdown anterior
result = df_steam_unique[mask] # resultado del filtrado.
print(f"Se han detectado {result.shape[0]} casos donde todas las columnas menos name y steam_appid tienen valores nulos.")
result.head()

Se han detectado 149 casos donde todas las columnas menos name y steam_appid tienen valores nulos.


,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,supported_languages,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,metacritic,reviews,categories,genres,recommendations,achievements,release_date,support_info,content_descriptors
26,Half-Life: Opposing Force,852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,Half-Life: Opposing Force,4330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,Half-Life: Opposing Force,8740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,Half-Life: Opposing Force,8955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336,Half-Life: Opposing Force,11610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Efectivamente, la sospecha se confirma. Podemos suponer que hubo algún error al extraer los datos, por lo cual prescindiremos de ellos porque son inservibles (tienen valores nulos en 30 de las 32 columnas).

In [13]:
df_clean = df_steam_unique.loc[~df_steam_unique.index.isin(result.index)]
print(f"El nuevo dataframe tiene {df_clean.shape[0]} filas.") # Tiene 149 filas menos que df_steam

El nuevo dataframe tiene 29079 filas.


Con esto afirmamos que no todos las filas llenas de valores nulos eran juegos duplicados. A continuación volveremos a ver los valores nulos por si podemos relacionar los duplicados con alguna columna.

In [14]:
print(df_clean.isnull().sum())

name                           1
steam_appid                    0
required_age                   0
is_free                        0
controller_support         23081
dlc                        24104
detailed_description          26
about_the_game                26
short_description             26
supported_languages           14
website                     9830
pc_requirements                0
mac_requirements               0
linux_requirements             0
legal_notice               19015
drm_notice                 28921
ext_user_account_notice    28567
developers                   115
publishers                     0
demos                      26940
price_overview              3563
packages                    3221
package_groups                 0
metacritic                 26104
reviews                    23174
categories                   565
genres                        47
recommendations            22355
achievements                2232
release_date                   0
support_in

Ahora nos queremos fijar en el precio de los videojuegos. El valor puede ser nulo en caso de que el juego sea gratuito. Para ver si coinciden los nulos de la columna *price_overview* con los juegos gratuitos del dataframe, ejecutaremos la siguiente celda:

In [15]:
juegos_gratuitos = df_clean.loc[df_clean["is_free"] & df_clean["price_overview"].isna()]
print(f"El dataframe juegos_gratuitos tiene {juegos_gratuitos.shape[0]} filas.")

El dataframe juegos_gratuitos tiene 2714 filas.


Parece ser que aún así hay videojuegos no gratuitos sin precio. Estos valores los reemplazaremos por el precio medio del dataframe, porque así estas filas nos servirán para las visualizaciones relacionadas con precios sin afectar gravemente el resultado. Siguiendo con los precios, ahora miraremos si todos los precios están en la misma divisa.

In [16]:
# Observaremos los distintos valores de divisa con un diccionario.
divisas = dict()
for row in range(df_clean.shape[0]):
    # El dataframe tiene estructuras de diccionarios pero en realidad son strings. Por eso tenemos que convertirlos a diccionario.
    try: # Hay valores nulos en la columna. Antes de que salga error, ignoramos la fila
        d = ast.literal_eval(df_clean.iloc[row]["price_overview"])    
    except:
        pass
    else:
        try:
            divisas[d['currency']]
        except:
            divisas[d['currency']]=True
    finally:
        row += 1

print(f"Las divisas en el dataframe son {list(el for el in divisas.keys())}.")

Las divisas en el dataframe son ['GBP', 'USD', 'EUR'].


Viendo que hay diferentes divisas, habrá que normalizar una de las tres. Escogeremos arbitrariamente EUR.

#### Conclusiones

- Viendo que no hay información respecto los DLCs y demos de los videojuegos, se simplificará la información de esas columnas.
- Hace falta cambiar muchos tipos de las columnas para que la estructura de datos sea más eficiente. Además, en las columnas **categories** y **genres** se puede almacenar la información en diccionarios externos y que el contenido de las columnas sea solamente una lista de números.
- La columna **is_free** nos será útil para determinar los precios de los juegos gratuitos a 0, y luego se eliminará la columna.
- Hay algunas columnas que contienen código HTML. Para ello, usaremos regex para quitar las etiquetas HTML.
